In [2]:
# 计算深度学习特征
# 参考https://github.com/AIM-Harvard/foundation-cancer-image-biomarker.git
import pandas as pd
import os
import SimpleITK as sitk

dataset_list = ["tjh"]

for dataset in dataset_list:
    data_path = f'./dataset/{dataset}/crop_image'
    mask_path = f'./dataset/{dataset}/crop_label'
    
    # 创建列表存储所有行数据
    rows = []

    for name in os.listdir(data_path):
        image_path = os.path.join(data_path, name)
        mask_file = os.path.join(mask_path, name)

        try:
            # 读取mask文件
            mask = sitk.ReadImage(mask_file)

            # 获取mask的中心点
            label_shape_filter = sitk.LabelShapeStatisticsImageFilter()
            label_shape_filter.Execute(mask)

            # 确保标签2存在
            if label_shape_filter.HasLabel(2):
                centroid = label_shape_filter.GetCentroid(2)
                x, y, z = centroid

                # 创建行数据
                row = {
                    "image_path": image_path,
                    "coordX": x,
                    "coordY": y,
                    "coordZ": z
                }
                rows.append(row)
            else:
                print(f"Warning: Label 2 not found in {name}")

        except Exception as e:
            print(f"Error processing {name}: {str(e)}")

    # 一次性创建DataFrame
    if rows:
        df = pd.DataFrame(rows)
        output_path = f'./coord_{dataset}.csv'
        df.to_csv(output_path, index=False)
        print(f"Successfully created {output_path}")
    else:
        print(f"No valid data found for dataset {dataset}")

Successfully created ./coord_tjh.csv


In [5]:
from monai.networks.nets import resnet50 as resnet50_monai
import torch
import torch.nn as nn

class LighterNet(nn.Module):
    def __init__(self, input_dim=2048):
        super(LighterNet, self).__init__()
        
        self.lighter = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 2)  # 假设最后输出2个类别
        )
        
        # 初始化权重
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0)

class CombinedNet(nn.Module):
    def __init__(self):
        super(CombinedNet, self).__init__()
        
        # ResNet50部分
        self.backbone = resnet50_monai(
            pretrained=False, 
            n_input_channels=1, 
            widen_factor=2, 
            conv1_t_stride=2, 
            feed_forward=False
        )
        
        # 加载预训练权重
        self.backbone.load_state_dict(
            torch.load('../foundation-cancer-image-biomarker/fmcib.torch')['trunk_state_dict']
        )
        
        # 冻结backbone参数
        for param in self.backbone.parameters():
            param.requires_grad = False
            
        # Lighter部分
        self.lighter = LighterNet()
        
    def forward(self, x):
        features = self.backbone(x)
        output = self.lighter(features)
        return output

# 实例化模型
model = CombinedNet()
device = torch.device('cuda:0')
model = model.to(device)

In [ ]:
from fmcib.run import get_features
import ssl

# 禁用SSL验证（注意：这种方式可能存在安全风险）
# ssl._create_default_https_context = ssl._create_unverified_context

feature_df = get_features("./coord_xiangyang.csv")